## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,42.46,93,75,4.61,broken clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,71.10,85,20,3.00,few clouds
2,2,Alto Araguaia,BR,-17.3147,-53.2153,67.95,98,100,7.16,overcast clouds
3,3,Inirida,CO,3.8653,-67.9239,70.20,63,36,2.75,scattered clouds
4,4,Pithora,IN,21.2667,82.5167,74.07,19,1,6.06,clear sky


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mar Del Plata,AR,-38.0023,-57.5575,71.10,85,20,3.00,few clouds
2,2,Alto Araguaia,BR,-17.3147,-53.2153,67.95,98,100,7.16,overcast clouds
3,3,Inirida,CO,3.8653,-67.9239,70.20,63,36,2.75,scattered clouds
4,4,Pithora,IN,21.2667,82.5167,74.07,19,1,6.06,clear sky
5,5,Port Elizabeth,ZA,-33.9180,25.5701,70.11,72,11,4.32,few clouds
6,6,Nicoya,CR,10.1483,-85.4520,78.35,77,9,6.82,clear sky
7,7,Itarema,BR,-2.9248,-39.9167,75.09,87,98,7.23,light rain
9,9,Cayenne,GF,4.9333,-52.3333,78.10,87,40,3.00,scattered clouds
11,11,Bakel,SN,14.9050,-12.4568,75.45,16,32,9.69,scattered clouds
14,14,Bredasdorp,ZA,-34.5322,20.0403,70.29,74,61,12.84,broken clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                283
City                   283
Country                278
Lat                    283
Lng                    283
Max Temp               283
Humidity               283
Cloudiness             283
Wind Speed             283
Current Description    283
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                278
City                   278
Country                278
Lat                    278
Lng                    278
Max Temp               278
Humidity               278
Cloudiness             278
Wind Speed             278
Current Description    278
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mar Del Plata,AR,71.10,few clouds,-38.0023,-57.5575,
2,Alto Araguaia,BR,67.95,overcast clouds,-17.3147,-53.2153,
3,Inirida,CO,70.20,scattered clouds,3.8653,-67.9239,
4,Pithora,IN,74.07,clear sky,21.2667,82.5167,
5,Port Elizabeth,ZA,70.11,few clouds,-33.9180,25.5701,
6,Nicoya,CR,78.35,clear sky,10.1483,-85.4520,
7,Itarema,BR,75.09,light rain,-2.9248,-39.9167,
9,Cayenne,GF,78.10,scattered clouds,4.9333,-52.3333,
11,Bakel,SN,75.45,scattered clouds,14.9050,-12.4568,
14,Bredasdorp,ZA,70.29,broken clouds,-34.5322,20.0403,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [46]:
# 7. Drop the rows where there is no Hotel Name.
clean = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = clean.dropna()
clean_hotel_df.count()

City                   123
Country                123
Max Temp               123
Current Description    123
Lat                    123
Lng                    123
Hotel Name             123
dtype: int64

In [47]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))